# Can't create file of 100 MiB on minIO space??

## Clear my space

In [1]:
import s3fs


In [2]:
access_key = !aws configure get aws_access_key_id
access_key = access_key[0]
secret_key = !aws configure get aws_secret_access_key
secret_key = secret_key[0]

In [3]:
client_kwargs={'endpoint_url': 'https://pangeo-eosc-minioapi.vm.fedcloud.eu/'}

#s3 = s3fs.S3FileSystem(anon=False, client_kwargs=client_kwargs) # Works only when using s3 in this Notebook, not with distributed.
s3 = s3fs.S3FileSystem(key=access_key, secret=secret_key, client_kwargs=client_kwargs)
s3.rm("s3://todaka-test/myfile")
s3.rm("s3://todaka-test/myfile_pr_an_limit")

s3.ls("s3://todaka-test/")


[]

## Create test dataset for writing

Change the value 
size_in_MiB 
for tests

In [12]:
import xarray as xr
import numpy as np

# Calculate the number of elements needed for "size_in_MiB" MiB of 64-bit floating point data
size_in_MiB  = 50
size_in_bytes = size_in_MiB * 1024 * 1024  # 30 MiB
bytes_per_element = 8  # 64-bit floating points
number_of_elements = 1024 // 8

# Create the NumPy array
data = np.random.rand(number_of_elements* 1024 * size_in_MiB)

# Choose dimensions that multiply together to equal the total number of elements
dim1, dim2, dim3 = size_in_MiB,  number_of_elements, 1024 #// (120 * 120)
#if dim1 * dim2 * dim3 != number_of_elements:
#    print 
#    raise ValueError("Chosen dimensions do not multiply to the total number of elements")

data = data.reshape((dim1, dim2, dim3))

# Create an xarray dataset
ds = xr.Dataset(
    {
        "variable": (("x", "y", "z"), data)
    },
    coords={
        "x": np.arange(dim1),
        "y": np.arange(dim2),
        "z": np.arange(dim3)
    }
)

# Save the dataset to a NetCDF file
ds.chunk('auto')



<xarray.Dataset>
Dimensions:   (x: 50, y: 128, z: 1024)
Coordinates:
  * x         (x) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
  * y         (y) int64 0 1 2 3 4 5 6 7 8 ... 120 121 122 123 124 125 126 127
  * z         (z) int64 0 1 2 3 4 5 6 7 ... 1017 1018 1019 1020 1021 1022 1023
Data variables:
    variable  (x, y, z) float64 dask.array<chunksize=(50, 128, 1024), meta=np.ndarray>

## Store the file in s3

In [13]:
target_root="s3://todaka-test/"
filepath=f"{target_root}test.zarr"

storage_options = {
    'anon': False, 
    'profile' : "default",
    'client_kwargs': {
        "endpoint_url": 'https://pangeo-eosc-minioapi.vm.fedcloud.eu/',
    }}

In [14]:
ds.chunk('auto').to_zarr(
#    'test.zarr', mode='w',compute=True)
    filepath, mode='w', 
    consolidated=True, 
    storage_options=storage_options)


## Results

| size | works? |
|------|--------|
| 100  | KO     |
| 70   | KO     |
| 50   | OK     |

In [16]:
storage_options = {
    'anon': False, 
    'profile' : "default",
    'client_kwargs': {
        "endpoint_url": 'https://pangeo-eosc-minioapi.vm.fedcloud.eu/',
    }}

xr.open_zarr(
    filepath,
    storage_options=storage_options)

<xarray.Dataset>
Dimensions:   (x: 50, y: 128, z: 1024)
Coordinates:
  * x         (x) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
  * y         (y) int64 0 1 2 3 4 5 6 7 8 ... 120 121 122 123 124 125 126 127
  * z         (z) int64 0 1 2 3 4 5 6 7 ... 1017 1018 1019 1020 1021 1022 1023
Data variables:
    variable  (x, y, z) float64 dask.array<chunksize=(50, 128, 1024), meta=np.ndarray>